In [81]:
from cycling_manager.preprocess import get_data, preprocess, split
from cycling_manager.new_sequences import get_num_sequence, get_images, get_full_sequence

In [76]:
df = preprocess(get_data())

full df shape (603619, 25)
full df shape (603619, 25)
Score vertical meters - model: DecisionTreeRegressor() score: 0.9995680099389864
Score vertical meters - model: DecisionTreeRegressor() score: 0.9999317297613699
Score vertical meters - model: KNeighborsClassifier() score: 0.9941044211660606


In [89]:
train, test = split(df, 2021, 2022)
train.head()

,name,year,race_name
0,tadej-pogacar,2021,tour-de-france
369,sepp-kuss,2021,vuelta-a-espana
363,mikel-landa,2021,giro-d-italia
364,krists-neilands,2021,giro-d-italia
365,rafal-majka,2021,vuelta-a-espana


In [90]:
X_decoder_num, y_decoder, X_encoder_num_no_y, y_encoder_num, X_decoder_img_ls, X_encoder_img_ls, y_encoder_img_ls=\
    get_num_sequence(df, 'magnus-cort-nielsen', 2021, "tour-de-france", 40, 30, img=True, binary=True, y_encoder=True)

In [91]:
X_decoder_num.shape, y_decoder.shape, X_encoder_num_no_y.shape, y_encoder_num.shape

(TensorShape([21, 5]),
 TensorShape([21, 1]),
 TensorShape([40, 14]),
 TensorShape([40, 1]))

In [112]:
X_encoder_img, X_decoder_img, y_encoder_img, to_drop_ls = get_images(X_decoder_img_ls, X_encoder_img_ls, y_encoder_img_ls, resnet=False)

In [114]:
len(X_encoder_img)

10

In [116]:
X_decoder_num_ls, y_decoder_ls, X_encoder_num_no_y_ls, y_encoder_num_ls, X_img_decoder_ls, X_img_encoder_ls, y_img_encoder_ls =\
    get_full_sequence(df, train, maxlen_num=40, maxlen_img=20, img=True, binary=True, y_encoder = True, num_cap=9999, img_cap=20, resnet=False)


 passed for ('krists-neilands', 2021, 'giro-d-italia') due to nan

 passed for ('manuel-belletti', 2021, 'giro-d-italia') due to nan


[ WARN:0@6991.586] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2021_stage-2.jpg'): can't open/read file: check file path/integrity
[ WARN:0@6991.586] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2021_stage-3.jpg'): can't open/read file: check file path/integrity
[ WARN:0@6991.891] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2020_stage-4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@6992.032] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-t


 passed for ('jacopo-mosca', 2021, 'giro-d-italia') due to nan


[ WARN:0@7000.453] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2020_stage-4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7000.810] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2020_stage-4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7006.058] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/nc-australia_2021_result.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7013.146] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/nc-australia_2021_resu


 passed for ('francesco-gavazzi', 2021, 'giro-d-italia') due to nan


[ WARN:0@7024.729] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2019_stage-4.jpg'): can't open/read file: check file path/integrity


(519, 21, 5) (519, 21, 1) (519, 40, 14) (519, 40, 1) (519, 21, 150, 300, 3) (519, 20, 150, 300, 3) (519, 20)


In [117]:
X_decoder_num_ls, y_decoder_ls, X_encoder_num_no_y_ls, y_encoder_num_ls, X_img_decoder_ls, X_img_encoder_ls, y_img_encoder_ls =\
    get_full_sequence(df, train, maxlen_num=40, maxlen_img=20, img=True, binary=True, y_encoder = True, num_cap=9999, img_cap=20, resnet=True)


 passed for ('krists-neilands', 2021, 'giro-d-italia') due to nan

 passed for ('manuel-belletti', 2021, 'giro-d-italia') due to nan


[ WARN:0@7036.272] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2021_stage-2.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7036.272] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2021_stage-3.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7036.570] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2020_stage-4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7036.719] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-t


 passed for ('jacopo-mosca', 2021, 'giro-d-italia') due to nan


[ WARN:0@7044.966] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2020_stage-4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7045.317] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2020_stage-4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7050.485] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/nc-australia_2021_result.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7057.491] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/nc-australia_2021_resu


 passed for ('francesco-gavazzi', 2021, 'giro-d-italia') due to nan


[ WARN:0@7069.231] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2019_stage-4.jpg'): can't open/read file: check file path/integrity


(519, 21, 5) (519, 21, 1) (519, 40, 14) (519, 40, 1) (519, 21, 150, 300, 3) (519, 20, 150, 300, 3) (519, 20)
